# LSTM CRF Results Analysis

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
folder_path = "results"

In [ ]:
loss_files = [f for f in os.listdir(folder_path) if f.startswith("losses") and f.endswith(".csv")]
accuracy_files = [f for f in os.listdir(folder_path) if f.startswith("accuracies") and f.endswith(".csv")]

In [ ]:
def extract_hyperparams(filename):
    parts = filename.split("_")
    lr = float(parts[2])
    dropout = float(parts[4].replace(".csv", ""))
    return lr, dropout

In [ ]:
loss_groups = {}
accuracy_groups = {}

In [ ]:
for file in loss_files:
    lr, dropout = extract_hyperparams(file)
    loss_groups[(lr, dropout)] = os.path.join(folder_path, file)

for file in accuracy_files:
    lr, dropout = extract_hyperparams(file)
    accuracy_groups[(lr, dropout)] = os.path.join(folder_path, file)

In [ ]:
for lr, dropout in loss_groups.keys():
    # Read the corresponding loss and accuracy files
    loss_df = pd.read_csv(loss_groups[(lr, dropout)])
    accuracy_df = pd.read_csv(accuracy_groups[(lr, dropout)])

    # Multiply index of val_loss by 50
    loss_df["epoch"] = loss_df["epoch"]
    loss_df["val_loss_epoch"] = ((loss_df["val_loss"].notnull()).cumsum() - 1) * 50 + 50
    loss_df["val_loss_epoch"] = loss_df["val_loss_epoch"].where(loss_df["val_loss"].notnull())
    accuracy_df["epoch"] = accuracy_df["epoch"] * 50

    # Create a figure with two subplots
    fig, axes = plt.subplots(2, 1, figsize=(10, 12))

    # Plot losses
    axes[0].plot(loss_df["epoch"], loss_df["train_loss"], label="Training Loss", color="blue")
    axes[0].plot(loss_df["val_loss_epoch"], loss_df["val_loss"], label="Validation Loss", color="orange")
    axes[0].set_title(f"Losses for LR: {lr}, Dropout: {dropout}")
    axes[0].set_xlabel("Epochs")
    axes[0].set_ylabel("Loss")
    axes[0].legend()
    axes[0].grid()

    # Plot accuracies
    axes[1].plot(accuracy_df["epoch"], accuracy_df["train_accuracy"], label="Training Accuracy", color="green")
    axes[1].plot(accuracy_df["epoch"], accuracy_df["val_accuracy"], label="Validation Accuracy", color="red")
    axes[1].set_title(f"Accuracies for LR: {lr}, Dropout: {dropout}")
    axes[1].set_xlabel("Epochs")
    axes[1].set_ylabel("Accuracy")
    axes[1].legend()
    axes[1].grid()

    # Adjust layout and save the figure
    plt.tight_layout()
    plot_filename = os.path.join(folder_path, f"plot_lr_{lr}_dropout_{dropout}.png")
    plt.savefig(plot_filename)
    plt.close()

    print(f"Saved plot for LR: {lr}, Dropout: {dropout} at {plot_filename}")